In [ ]:
import open3d as o3d
import torch
from torch.utils.data import DataLoader

from gradslam import RGBDImages
from gradslam.slam import PointFusion, ICPSLAM
from gradslam.datasets import ICL, TUM

In [ ]:
# Dataset to use
dataset_name = "icl"  # "icl", "tum"

# Path to the dataset directory
dataset_path = "G:\Datasets\ICL"

# Odometry method to use
odometry = "gt"  # "gt", "icp", "gradicp"

# select device
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
# load dataset
Dataset = ICL if dataset_name == "icl" else TUM
dataset = Dataset(dataset_path, seqlen=20, height=240, width=320)
loader = DataLoader(dataset=dataset, batch_size=2)
colors, depths, intrinsics, poses, *_ = next(iter(loader))

# create rgbdimages object
rgbdimages = RGBDImages(colors, depths, intrinsics, poses, channels_first=False)

# visualize rgbdimages
fig1 = rgbdimages.plotly(0)
fig1.update_layout(autosize=False, height=720, width=500).show()

In [ ]:
# SLAM
slam = PointFusion(odom=odometry, dsratio=4, device=device)
pointclouds, recovered_poses = slam(rgbdimages)

In [ ]:
# visualize generated pointcloud map
# Plotly visualization
fig2 = pointclouds.plotly(0).update_layout(autosize=False, height=600)
fig2.show()

# Open3d visualization
# o3d.visualization.draw_geometries([pointclouds.open3d(0)])